In [6]:
from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader
import gradio as gr
import os

In [2]:
load_dotenv(override=True)

True

In [4]:
openai = OpenAI()

In [10]:
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("OPENAI_API_KEY is not set in the environment variables.") 
else:
    print("API key set")

API key set


In [16]:
# Function to read PDF and extract text
reader = PdfReader("info folder/profile.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

In [17]:
#opening the summary.rtf file
with open("info folder/summary.rtf", "r") as file:
    summary = file.read()

In [ ]:
#check if the summary is read correctly
print(summary)

{\rtf1\ansi\ansicpg1252\cocoartf2821
\cocoatextscaling0\cocoaplatform0{\fonttbl\f0\fswiss\fcharset0 Helvetica;}
{\colortbl;\red255\green255\blue255;}
{\*\expandedcolortbl;;}
\paperw11900\paperh16840\margl1440\margr1440\vieww11520\viewh8400\viewkind0
\pard\tx720\tx1440\tx2160\tx2880\tx3600\tx4320\tx5040\tx5760\tx6480\tx7200\tx7920\tx8640\pardirnatural\partightenfactor0

\f0\fs24 \cf0 \
\
### **Education**\
\
* **Schooling (1st\'9610th Standard):**\
  Attended **Oxford English School**, Bangalore.\
  Secured **88%** in the 10th standard (SSLC), which is considered a **distinction**.\
\
* **Pre-University (PU College):**\
  Studied at **The Oxford PU College**, Bangalore.\
  Graduated with **66%**, categorized as a **second class**.\
\
* **Undergraduate Degree:**\
  Completed a **Bachelor of Arts (B.A.)** specializing in **Psychology** from **University of Mysore**, Mysore.\
  The degree was pursued through **distance education / correspondence mode**.\
\
* **Postgraduate Education:**\
  

In [23]:
#setting the system prompt
name = "Deekshith Someshwar"
system_prompt = f"You are acting as {name}. You are going to answer question on {name}'s personal bio. \
You will be given a summary of {name}'s personal bio and a LinkedIn profile. You will answer the question \
based on the summary and LinkedIn profile. Your responsibility is to answer questions about {name} \
as professionally and accurately as possible. You are given {name}'s summary and linkedin profile, and its  \
your responsibility to answer questions about {name} as professionally and accurately as possible. \
You will be given a question and you will answer the question based on the summary and LinkedIn profile. \
You will politely decline to answer {name}'s personal information other than what is provided and mention this is a professional \
bio and to reach out to me if they wish to know me personally"
system_prompt += f"\n\nSummary:\n{summary}\n\nLinkedIn Profile:\n{linkedin}"    
system_prompt += "\n\nYou will be given a question and you will answer the question based on the summary and LinkedIn profile."


In [31]:
def chat(message, history):
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": message}
    ]
    response = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
    )

    return response.choices[0].message.content
    

In [ ]:
# Create a Gradio chat interface
gr.ChatInterface(chat, type= "messages").launch()

In [33]:
#Adding a pydantic model to validate the output and come up with better output in case of errors
from pydantic import BaseModel

In [34]:
class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str

In [40]:
evaluator_system_prompt = f"You are an evaluator. You will evaluate the output of the chat interface and decide if \
the output is acceptable or not. You will also provide feedback on the output. \
The agent is playing the role of {name} and is given to responsibility to answer questions about {name}'s personal bio. \
You will be given the output of the chat interface and you will evaluate the output based on the following criteria: \
1. The output should be relevant to the question asked. \
2. The output should be professional and accurate. \
3. The output should not contain any personal information other than what is provided in the summary and LinkedIn profile. \
4. The output should be polite and respectful. \
5. The output should not contain any offensive or inappropriate content. "
evaluator_system_prompt += f"\n\n you will be given the {summary} and {linkedin} to evaluate the output. "
evaluator_system_prompt += "\n\n With this context, you need to decide if the output is acceptable or not, and suggest to make changes \
    if it is not acceptable. "

In [41]:
def evaluator_user_prompt(reply, message, history):
    user_prompt = f" Here is the output of the chat interface:\n\n{reply}\n\nQuestion: {message}\n\nHistory: {history}"
    return user_prompt

In [42]:
def evaluate(reply, message, history) -> Evaluation:
    messages = [
        {"role": "system", "content": evaluator_system_prompt},
        {"role": "user", "content": evaluator_user_prompt(reply, message, history)}
    ]
    response = openai.beta.chat.completions.parse(
        model="gpt-4o-mini",
        messages=messages,
        response_format = Evaluation
    )

    return response.choices[0].message.content

In [43]:
def rerun(reply, message, history, feedback):
    updated_system_prompt = system_prompt + f"\n\nResponse rejected by the evaluator: {feedback}"
    messages = [
        {"role": "system", "content": updated_system_prompt},
        {"role": "user", "content": message}
    ]
    response = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
    )   
    return response.choices[0].message.content

In [46]:
def chat(message, history):
    messages = [
        {"role": "system", "content": system_prompt}, 
        {"role": "user", "content": message}
        ]
    response = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
    )
    reply = response.choices[0].message.content
    evaluation = evaluate(reply, message, history)

    if evaluation.is_acceptable:
        print("Output is acceptable" )
    else:
        print("Output is not acceptable. Feedback: ", evaluation.feedback)
        reply = rerun(reply, message, history, evaluation.feedback)
    return reply

In [47]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7866
* To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "/Users/deekshithsomeshwar/Documents/projects/agents/.venv/lib/python3.12/site-packages/gradio/queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/deekshithsomeshwar/Documents/projects/agents/.venv/lib/python3.12/site-packages/gradio/route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/deekshithsomeshwar/Documents/projects/agents/.venv/lib/python3.12/site-packages/gradio/blocks.py", line 2220, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/deekshithsomeshwar/Documents/projects/agents/.venv/lib/python3.12/site-packages/gradio/blocks.py", line 1729, in call_function
    prediction = await fn(*processed_input)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/deekshith